In [1]:

import math, json, re, ccxt
from pprint import pformat
wex = ccxt.wex()
data_for_graph = {}
data = wex.fetch_tickers()
for key in data:
    data_for_graph.update({key: data[key]['last']})
ticker_list = list(data_for_graph.keys())
ticker_list2 = []
for key in ticker_list:
    ticker_list2.append("{1}/{0}".format(*key.split('/'))) #обратные тикеры

for key in range(0, len(ticker_list)):
    #обратные котировки
    data_for_graph.update({ticker_list2[key]: 1 / (data[ticker_list[key]]['last'])})

print(pformat(data_for_graph))


{'BCH/BCHET': 1.51285930408472,
 'BCH/BTC': 0.1123,
 'BCH/DASH': 2.419,
 'BCH/ETH': 1.938,
 'BCH/EUR': 749.246,
 'BCH/LTC': 6.309,
 'BCH/RUR': 52465.474,
 'BCH/USD': 921.86,
 'BCH/ZEC': 4.12,
 'BCHET/BCH': 0.661,
 'BTC/BCH': 8.904719501335709,
 'BTC/BTCET': 1.5432098765432098,
 'BTC/DASH': 21.547080370609784,
 'BTC/ETH': 17.22949689869056,
 'BTC/EUR': 6680.5582,
 'BTC/LTC': 56.433408577878104,
 'BTC/NMC': 5000.0,
 'BTC/NVC': 2173.913043478261,
 'BTC/PPC': 4347.826086956522,
 'BTC/RUR': 468320.45009,
 'BTC/USD': 8212.0,
 'BTC/ZEC': 36.76470588235294,
 'BTCET/BTC': 0.648,
 'DASH/BCH': 0.41339396444811904,
 'DASH/BTC': 0.04641,
 'DASH/DSHET': 1.3908205841446455,
 'DASH/ETH': 0.801,
 'DASH/EUR': 309.911,
 'DASH/LTC': 2.605,
 'DASH/RUR': 21685.71,
 'DASH/USD': 380.97871,
 'DASH/ZEC': 1.704,
 'DSHET/DASH': 0.719,
 'ETH/BCH': 0.5159958720330238,
 'ETH/BTC': 0.05804,
 'ETH/DASH': 1.2484394506866416,
 'ETH/ETHET': 1.6447368421052633,
 'ETH/EUR': 387.06718,
 'ETH/LTC': 3.26496,
 'ETH/RUR': 27014

Создаем граф в виде списка смежности для каждой ноды - тикера криптовалюты, 
котировки переводим в логарифмический вид.

In [3]:
graph = {}
jsrates = data_for_graph

pattern = re.compile("([A-Z]{3,5})/([A-Z]{3,5})")

try:
    for key in jsrates:
        matches = pattern.match(key)

        conversion_rate = math.log(float(jsrates[key]))
        
        from_rate = matches.group(1).encode('ascii', 'ignore')

        to_rate = matches.group(2).encode('ascii', 'ignore')

        if from_rate != to_rate:
            if from_rate not in graph:
                graph[from_rate] = {}
            graph[from_rate][to_rate] = float(conversion_rate)

except AttributeError:
    print('Strange rate')
    pass

print(pformat(graph))
# воплощение алгоритма отсюда
#  https://gist.github.com/joninvski/701720


{b'BCH': {b'BCHET': 0.4140014391304507,
          b'BTC': -2.18658141723774,
          b'DASH': 0.8833542316278902,
          b'ETH': 0.6616565134685745,
          b'EUR': 6.6190673675105165,
          b'LTC': 1.8419771853886724,
          b'RUR': 10.867910594145691,
          b'USD': 6.8263933682093665,
          b'ZEC': 1.415853163361435},
 b'BCHET': {b'BCH': -0.41400143913045073},
 b'BTC': {b'BCH': 2.18658141723774,
          b'BTCET': 0.4338645826298623,
          b'DASH': 3.0702403257290527,
          b'ETH': 2.84662285096615,
          b'EUR': 8.80695682591376,
          b'LTC': 4.033061333805202,
          b'NMC': 8.517193191416238,
          b'NVC': 7.6842840684811335,
          b'PPC': 8.377431249041079,
          b'RUR': 13.056908062998875,
          b'USD': 9.013351778138825,
          b'ZEC': 3.6045383056801854},
 b'BTCET': {b'BTC': -0.4338645826298623},
 b'DASH': {b'BCH': -0.8833542316278903,
           b'BTC': -3.0702403257290527,
           b'DSHET': 0.32989392126109046,

In [4]:


# Step 1: For each node prepare the destination and predecessor
def initialize(graph, source):
    d = {}  # Stands for destination
    p = {}  # Stands for predecessor
    for node in graph:
        d[node] = float('Inf')  # We start admiting that the rest of nodes are very very far
        p[node] = None
    d[source] = 0  # For the source we know how to reach
    return d, p


def relax(node, neighbour, graph, d, p):
    # If the distance between the node and the neighbour is lower than the one I have now
    if d[neighbour] > d[node] + graph[node][neighbour]:
        # Record this lower distance
        d[neighbour] = d[node] + graph[node][neighbour]
        p[neighbour] = node


def retrace_negative_loop(p, start):
    arbitrageLoop = [start]
    next_node = start
    while True:
        next_node = p[next_node]
        if next_node not in arbitrageLoop:
            arbitrageLoop.append(next_node)
        else:
            arbitrageLoop.append(next_node)
            arbitrageLoop = arbitrageLoop[arbitrageLoop.index(next_node):]
            return arbitrageLoop


def bellman_ford(graph, source):
    d, p = initialize(graph, source)
    for i in range(len(graph) - 1):  # Run this until is converges
        for u in graph:
            for v in graph[u]:  # For each neighbour of u
                relax(u, v, graph, d, p)  # Lets relax it

    # Step 3: check for negative-weight cycles
    for u in graph:
        for v in graph[u]:
            if d[v] < d[u] + graph[u][v]:
                return (retrace_negative_loop(p, source))
    return None


In [6]:
 paths = []
 for key in graph:

    path = bellman_ford(graph, key)

    if path not in paths and not None and len(path)>3:
        paths.append(path)

for path in paths:
    if path == None:
        print("No opportunity here :(")
    else:
        
        money = 100
        print("Starting with %(money)i in %(currency)s" % {"money": money, "currency": path[0]})           

        for i, value in enumerate(path):
            if i + 1 < len(path):
                start = path[i]
                end = path[i + 1]
                rate = math.exp(graph[start][end])
                money *= rate*0.998
                print("%(start)s to %(end)s at %(rate)f = %(money)f" % {"start": start, "end": end, "rate": rate,
                                                                        "money": money})
print ("\n")

 

Starting with 100 in b'BTC'
b'BTC' to b'NMC' at 5000.000000 = 499000.000000
b'NMC' to b'USD' at 1.662000 = 827679.324000
b'USD' to b'BTC' at 0.000122 = 100.587429


